In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import numpy as np
from astropy.io import ascii
from astropy import units as u
from lmfit.models import PolynomialModel, GaussianModel
from lmfit import Model as mod
from lmfit import minimize, Parameters, report_fit
import ccdproc as ccdp
from pathlib import Path
import os
import glob
import shutil

In [10]:
os.makedirs("OHbaseline plots", exist_ok=True)

In [3]:
# Baseline + Baseline Subtraction Graphs
# Load all files matching pattern
file_list = sorted(glob.glob('*.ascii'))  # Assumes files are in the same folder as this script

# Process each file
for filename in file_list:
    data = np.loadtxt(filename, skiprows = 3)
    filename_base = os.path.splitext(os.path.basename(filename))[0]
    x = data[:, 0]
    y = data[:, 1]

    ### Polynomial Baseline Fit ###
    poly_model = PolynomialModel(degree=4)
    gooddata1 = list(np.where((x >= -30) & (x <= -15))[0])
    gooddata2 = list(np.where((x >= 15) & (x <= 30))[0])
    gooddata = gooddata1 + gooddata2

    params = poly_model.guess(y[gooddata], x=x[gooddata])
    result = poly_model.fit(y[gooddata], params, x=x[gooddata])
    y_eval = poly_model.eval(result.params, x=x)

    ### Plotting Original with Baseline ###
    plt.figure(figsize=(7, 5))
    plt.xlim([-30, 30])
    plt.ylim([-.50, .50])
    plt.axhline(0, color='green')
    plt.plot(x, y, label='Data')
    plt.plot(x, y_eval, label='Polynomial Baseline', color='red', linestyle='--')
    plt.xlabel('Radio(km/s)')
    plt.ylabel('TA(I)')
    plt.title(f'Baseline of OHData; {filename_base}')
    plt.legend()
    plt.tight_layout()
    filename_base = os.path.splitext(os.path.basename(filename))[0]
    plt.savefig(f"OHbaselineplots/baseline_{filename_base}.png")
    plt.close()

    ### Subtract Baseline ###
    y_sub = y - y_eval

    ### Plotting Subtracted Data ###
    plt.figure(figsize=(7, 5))
    plt.xlim([-30, 30])
    plt.ylim([-.50, .50])
    plt.axhline(0, color='green')
    plt.plot(x, y, label='Data')
    plt.plot(x, y_sub, label='Subtracted Polynomial Baseline', color='red', linestyle='--')
    plt.xlabel('Radio(km/s)')
    plt.ylabel('TA(I)')
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    plt.title(f'Subtracted Baseline of OHData; {filename_base}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"OHbaselineplots/subtracted_baseline_{filename_base}.png")
    plt.close()
print("All 72 baseline files processed.")

All 72 baseline files processed.


In [15]:
print(file_list)

['158375n19250_65_ii.ascii', '158375n19250_67_ii.ascii', '158500n19250_65_ii.ascii', '158500n19250_67_ii.ascii', '158500n19375_65_ii.ascii', '158500n19375_67_ii.ascii', '158625n19375_65_ii.ascii', '158625n19375_67_ii.ascii', '158750n19375_65_ii.ascii', '158750n19375_67_ii.ascii', '158750n19500_65_ii.ascii', '158750n19500_67_ii.ascii', '158875n19500_65_ii.ascii', '158875n19500_67_ii.ascii', '158875n19625_65_ii.ascii', '158875n19625_67_ii.ascii', '159000n19625_65_ii.ascii', '159000n19625_67_ii.ascii', '159000n19750_65_ii.ascii', '159000n19750_67_ii.ascii', '159125n19750_65_ii.ascii', '159125n19750_67_ii.ascii', '159125n19875_65_ii.ascii', '159125n19875_67_ii.ascii', '159250n19875_65_ii.ascii', '159250n19875_67_ii.ascii', '159250n20000_65_ii.ascii', '159250n20000_67_ii.ascii', '159375n20000_65_ii.ascii', '159375n20000_67_ii.ascii', '159500n20000_65_ii.ascii', '159500n20000_67_ii.ascii', '159500n20125_65_ii.ascii', '159500n20125_67_ii.ascii', '159625n20125_65_ii.ascii', '159625n20125_67_ii

In [13]:
print(os.getcwd())

C:\Users\dhami\Research\STARTastro\DhamiBusch-Research25\OHData


In [20]:
### Specify the current directory name and the new name##
# current_name = "plots"
# new_name = "OHbaselineplots"
# # Rename the directory
# os.rename(current_name, new_name)
# print(f"Directory renamed from '{current_name}' to '{new_name}'")

Directory renamed from 'plots' to 'OHbaselineplots'


In [22]:
### Measure RMS ###
# RMS red
file_list = sorted(glob.glob('*.ascii'))  # Assumes files are in the same folder as this script
# Store results
rms_data = []
for filename in file_list:
    # Load your data from ASCII
    data = np.loadtxt(filename, skiprows = 3)  # whichever works: skiprows, delimiter
    y = data[:, 1]
    y_sub = y - y_eval
    gooddata1 = list(np.where((x >= -30) & (x <= -15))[0])
    gooddata2 = list(np.where((x >= 15) & (x <= 30))[0])
    gooddata = gooddata1 + gooddata2
    noise=np.std(y[gooddata]) * u.K
    noise_sub=np.std(y_sub[gooddata]) * u.K
        # Save results
    rms_data.append({
        "filename": filename,
        "noise(K)": noise.value,
        "noise_sub(K)": noise_sub.value
    })
# Convert into pandas
df = pd.DataFrame(rms_data)
print(df)

                    filename  noise(K)  noise_sub(K)
0   158375n19250_65_ii.ascii  0.010751      0.003011
1   158375n19250_67_ii.ascii  0.010751      0.003011
2   158500n19250_65_ii.ascii  0.006266      0.004192
3   158500n19250_67_ii.ascii  0.006266      0.004192
4   158500n19375_65_ii.ascii  0.007098      0.003636
5   158500n19375_67_ii.ascii  0.007098      0.003636
6   158625n19375_65_ii.ascii  0.011241      0.003412
7   158625n19375_67_ii.ascii  0.011241      0.003412
8   158750n19375_65_ii.ascii  0.008157      0.002502
9   158750n19375_67_ii.ascii  0.008157      0.002502
10  158750n19500_65_ii.ascii  0.009377      0.003022
11  158750n19500_67_ii.ascii  0.009377      0.003022
12  158875n19500_65_ii.ascii  0.008275      0.002710
13  158875n19500_67_ii.ascii  0.008275      0.002710
14  158875n19625_65_ii.ascii  0.019353      0.010410
15  158875n19625_67_ii.ascii  0.019353      0.010410
16  159000n19625_65_ii.ascii  0.008222      0.003056
17  159000n19625_67_ii.ascii  0.008222      0.

In [23]:
# Display all columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', None)        # Prevent line wrapping
print(df)

                    filename  noise(K)  noise_sub(K)
0   158375n19250_65_ii.ascii  0.010751      0.003011
1   158375n19250_67_ii.ascii  0.010751      0.003011
2   158500n19250_65_ii.ascii  0.006266      0.004192
3   158500n19250_67_ii.ascii  0.006266      0.004192
4   158500n19375_65_ii.ascii  0.007098      0.003636
5   158500n19375_67_ii.ascii  0.007098      0.003636
6   158625n19375_65_ii.ascii  0.011241      0.003412
7   158625n19375_67_ii.ascii  0.011241      0.003412
8   158750n19375_65_ii.ascii  0.008157      0.002502
9   158750n19375_67_ii.ascii  0.008157      0.002502
10  158750n19500_65_ii.ascii  0.009377      0.003022
11  158750n19500_67_ii.ascii  0.009377      0.003022
12  158875n19500_65_ii.ascii  0.008275      0.002710
13  158875n19500_67_ii.ascii  0.008275      0.002710
14  158875n19625_65_ii.ascii  0.019353      0.010410
15  158875n19625_67_ii.ascii  0.019353      0.010410
16  159000n19625_65_ii.ascii  0.008222      0.003056
17  159000n19625_67_ii.ascii  0.008222      0.

In [9]:
# Computing the Integral
def integral(vel, ta, bounds, deltav):
    mask = (vel >= bounds[0]) & (vel <= bounds[1])
    deltav = vel[1]-vel[0]
    return np.sum(ta[mask])*deltav
# Set the path to your ASCII files
file_list = sorted(glob.glob('*.ascii'))  # Assumes files are in the same folder as this script
bounds = [-15, 15]
# Process each file
for filename in file_list:
    data = np.loadtxt(filename, skiprows=3)
    vel = data[:, 0]
    ta = data[:, 1]
    deltav = vel[1]-vel[0]
    # Baseline subtraction
    poly_model = PolynomialModel(degree=4)
    gooddata = np.where((vel >= -15) & (vel <= 15))[0]
    params = poly_model.guess(ta[gooddata], x=vel[gooddata])
    result = poly_model.fit(ta[gooddata], params, x=vel[gooddata])
    y_eval = poly_model.eval(result.params, x=vel)
    ta = y - y_eval
    # Compute integral
    result = integral(vel, ta, bounds, deltav)
    print(f"{filename}: Integral from {bounds[0]} to {bounds[1]} = {result}")

158375n19250_65_ii.ascii: Integral from -15 to 15 = 0.3152890863854345
158375n19250_67_ii.ascii: Integral from -15 to 15 = 0.8128149339598133
158500n19250_65_ii.ascii: Integral from -15 to 15 = 0.6324137438983927
158500n19250_67_ii.ascii: Integral from -15 to 15 = 0.39010851198215624
158500n19375_65_ii.ascii: Integral from -15 to 15 = 0.41701803192813086
158500n19375_67_ii.ascii: Integral from -15 to 15 = 0.27700299315034244
158625n19375_65_ii.ascii: Integral from -15 to 15 = -0.3830175054711625
158625n19375_67_ii.ascii: Integral from -15 to 15 = -0.1524554362380025
158750n19375_65_ii.ascii: Integral from -15 to 15 = -0.08201852197462046
158750n19375_67_ii.ascii: Integral from -15 to 15 = -0.031606283924659404
158750n19500_65_ii.ascii: Integral from -15 to 15 = 1.0059311022243276
158750n19500_67_ii.ascii: Integral from -15 to 15 = 2.060113209308459
158875n19500_65_ii.ascii: Integral from -15 to 15 = 0.5309112899498559
158875n19500_67_ii.ascii: Integral from -15 to 15 = 0.40322285194944